# Bayesian Optimization of Fast Charging Protocols

This notebook demonstrates the use of the Ax Bayesian Optimization framework to tune a multi-step CC-CV fast charging protocol for a Li-ion battery simulated using a P2D cell model (w/ plating) in PyBaMM. Essentially, this project seeks to answer the question: "What is the charging protocol that maximizes charge in 30 minutes while minimizing aging to the battery?"

---

## Background

We now start with a 5-step charging protocol:  
- Step 1: C-rate C1 for 6 minutes 
- Step 2: C-rate C2 for 6 minutes 
- Step 3: C-rate C3 for 6 minutes 
- Step 4: C-rate C4 for 6 minutes
- Step 5: C-rate C5 for 6 minutes

where C1, C2, C3, C4, and C5 are the parameters to be optimized.


The optimization objective is: 

objective = $Q_{30} - \beta*\log(Q_{lost})$

where: 
- $Q_{30}$ is the charge stored in 30 minutes \[Ah\] 
- $Q_{lost}$ is the capacity lost due to lithium plating \[Ah\] 
- $\beta$ is a dimensionless weight that tunes how much the user wants to target high capacity versus low degradation 

This objective is chosen because it satisfies the following criteria:
1. Rewards high capacity
2. Penalizes lithium plating
3. Tunable
4. Stable

## Load Libraries

In [1]:
#PyBamm Library
import pybamm

#Numpy Library
import numpy as np

#Ax Libraries
from ax.api.client import Client
from ax.api.configs import  RangeParameterConfig

---

## Define the Simulation Function

In this section, we define the `run_P2D` function, which simulates a given charging protocol and returns the optimization objective. 

We use the DFN model with reversible plating enabled (based on LG M50 cell parameters) and evaluate performance after 30 minutes of charge. Before the charge cycle, the cell is discharged to it's minimum voltage at 1C.  


In [4]:
def run_P2D(params,beta):
    C1, C2, C3, C4, C5 = params[0], params[1], params[2], params[3], params[4]
    
    #Define DFN model with lihtium plating
    model = pybamm.lithium_ion.DFN(options={"lithium plating": "reversible"})
    
    #Define CC Experiment
    experiment = pybamm.Experiment([
        "Discharge at 1C until 2.6V",
        "Hold at 2.6V for 10 minutes",
        f"Charge at {C1}C for 6 minutes",
        f"Charge at {C2}C for 6 minutes",
        f"Charge at {C3}C for 6 minutes",
        f"Charge at {C4}C for 6 minutes",
        f"Charge at {C5}C for 6 minutes",
    ])
    pv = pybamm.ParameterValues("OKane2022")
    
    #Run Simulation
    sim = pybamm.Simulation(model, experiment=experiment,parameter_values=pv)
    sim.solve(solver=pybamm.CasadiSolver(mode="safe", dt_max=1))
    
    #Obtain Simulation Observables
    time_sim = sim.solution["Time [min]"].data
    lost_capacity = np.max(sim.solution['Loss of capacity to negative lithium plating [A.h]'].data)
    
    #Calculate Amount of Charge in 30 minutes
    I = sim.solution["Current [A]"].data          #store current data
    t0 = time_sim[np.where(I < 0)[0][0]]     #index first charging point
    charging_mask = (I < 0) & (time_sim <= t0 + 1800) #define 30 minute charging range
    Q30 = np.trapz(-I[charging_mask], time_sim[charging_mask])   #coulomb count for 30 minute charging range
    Q30_Ah = Q30 / 3600     #convert to Ah
    
    objective = Q30_Ah - beta*np.log(lost_capacity)
    
    print({"lost_capacity": lost_capacity, "Q30": Q30_Ah, "objective": objective})
    return {"objective": objective}


---

## Test Simulation Weight

Before optimizing, it's important to test the P2D model to ensure that the simulation returns sensical values and that the weighting factor ($\beta$) reflects the goals of the user (ex: higher capacity vs less plating). Since the parameters of the P2D model are based on an NMC-graphite chemistry (i.e relatively stable so plating shouldn't be very destructive), I will target a higher capacity. The following ranking is desired: 

Rank 1 (Worst): Low capacity + High degradation \
Rank 2: Low capacity + Low degradation \
Rank 3: High capacity + High degradation \
Rank 4 (Best): High capacity + Low degradation

If a more passive protocol is desired (i.e less degradation is favored), then ranks 2 and 3 could be switched as is done in the "3step" notebook.


In [6]:
beta = 0.008 #beta = 0.011 is very balanced but slightly on less degradation side (passive) | beta = 0.008 is slightly on more capacity side (aggressive)
run_P2D([0.25,0.25,0.25,0.25,0.25],beta)
run_P2D([0.5,0.5,0.5,0.5,0.5],beta)
run_P2D([2,2,2,2,2],beta)
run_P2D([3,2.5,2,1.5,1],beta) 

At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0005620043797975666, 'Q30': 0.010416666666666663, 'objective': 0.07028867398560143}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0006277617094485365, 'Q30': 0.020833333333333325, 'objective': 0.07982013258978359}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.4483237218600773, 'Q30': 0.0833333333333333, 'objective': 0.08975125104592867}


At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.39359550475025973, 'Q30': 0.0833333333333333, 'objective': 0.090792785611152}


{'objective': 0.090792785611152}

### Test Results

Two things to note after this initial test:

1. The P2D + plating model is indeed returning values that make physical sense. As the C-rate is doubled from C/4 -> C/2, the amount of charge is doubled but the capacity lost to plating is nearly the same. However, at higher C-rates (ex: 2C) there is significant plating as expected. Finally, when the current starts high but is decreased as the charge progresses (a common strategy used in fast charging), the degree of plating decreases since lithium intercalation is easiest at low SOC. 


2. $\beta$ = 0.008 results in the ranking that I desired, but again this can be readily changed to reflect a more passive strategy that favors less degradation.

---



## Run Bayesian Optimization

Ax is now called to optimize the charge profile (C1, C2, C3, C4, C5). By default, the algorithm uses Gaussian Process (GP) as the surrogate model and Expected Improvement (EI) as the utiliy function. 

### Initialize Algorithm
The following cell initializes the experiment, defines input parameters, and sets the optimization objective. The maximum C-rate is 3C to avoid Vmax from being reached, and the objective is maximized. 


In [ ]:
#1. Initialize Client for Experiment
client = Client()

#2. Define Input Parameter Name, Type, and Bounds/Values
C1 = RangeParameterConfig(name="C1", parameter_type="float", bounds=(0, 3))   #3C is max to prevent Vmax from being reached
C2 = RangeParameterConfig(name="C2", parameter_type="float", bounds=(0, 3))
C3 = RangeParameterConfig(name="C3", parameter_type="float", bounds=(0, 3))
C4 = RangeParameterConfig(name="C4", parameter_type="float", bounds=(0, 3))
C5 = RangeParameterConfig(name="C5", parameter_type="float", bounds=(0, 3))

#3. Configure Experiment
client.configure_experiment(
    parameters=[C1,C2,C3,C4,C5],
    # The following arguments are only necessary when saving to the DB
    name="Fast-Charging-Exp",
    description="Optimize fast charging profile-aggressive",
)

#4. Set Optimization Objective
client.configure_optimization(objective="objective")

### Attach Preexisting Trials

Attach the data from the inital simulation test to train the initial surrogate model.

In [ ]:
# Pairs of previously evaluated parameterizations and associated metric readings
preexisting_trials = [
    (
        {"C1": 0.25, "C2": 0.25, "C3": 0.25, "C4": 0.25, "C5": 0.25},
        {"objective": 0.07028867398560143},
    ),
    (
        {"C1": 0.5, "C2": 0.5, "C3": 0.5, "C4": 0.5, "C5": 0.5},
        {"objective": 0.07981729329398196},
    ),
    (
        {"C1": 2.0, "C2": 2.0, "C3": 2.0, "C4": 2.0, "C5": 2.0},
        {"objective": 0.08975124385218475},
    ),
    (
        {"C1": 3.0, "C2": 2.5, "C3": 2.0, "C4": 1.5, "C5": 1.0},
        {"objective": 0.090792785611152},
    ),

]

for parameters, data in preexisting_trials:
    # Attach the parameterization to the Client as a trial and immediately complete it with the preexisting data
    trial_index = client.attach_trial(parameters=parameters)
    client.complete_trial(trial_index=trial_index, raw_data=data)

### Run Optimization

Every epoch, obtain 3 new charge profile recommendations and test them using the `run_P2D` function for N epochs (N = 20 takes about 5 minutes). 

In [ ]:
N = 40 #number of BO epochs
for i in range(N):
    
    #Get new trials
    trials = client.get_next_trials(max_trials=3)
    
    # Tell Ax the result of those trials
    for trial_index, parameters in trials.items():
        client.complete_trial(trial_index=trial_index, raw_data=run_P2D(list(parameters.values()),beta))

### Analyze Results

Print the best predicted paramters and show the detailed analysis from the BO algorithm

In [ ]:
best_parameters, prediction, index, name = client.get_best_parameterization()
print("Best Parameters:", best_parameters)
print("Prediction (mean, variance):", prediction)

cards = client.compute_analyses(display=True)

---

## Results & Discussion 

After running this process 10 times, the most optimal charge profile the BO algorithm returned was:

1. C1 = 3C (6 min)
2. C2 = 0C (6 min)
3. C3 = 0C (6 min)
4. C4 = 0C (6 min)
5. C5 = 2.52C (6 min)

lost_capacity: 0.048 | Q30: 0.115 | objective: 0.139 

This is quite similar to the 3-step optimal solution where a rest period is used between charging to allow the cell to relax thus minimizing degradation. However, this 5-step profile resulted in a slightly better objective than the 3-step one (0.139 vs 0.123) due to significantly less capacity loss with similar amounts of charge capacity; most likely caused by the longer rest periods used.

Unlike the 3-step optimization, the 5-step one suggested a different charge profile all 10 times that it was ran. Interestingly, every suggested charge profile included at least 1 rest period; exemplifying the importance of rest to minimze degradation. Although the exact charge currents were different, some solutions had very strong similarities at least qualitatively. Out of the 10 suggestions, two strategies were proposed the most often (4 times each) and here are the results of a representative sample from each :

1. **"Long Rest" (Optimal)**: Charge (6 minutes) - Rest (18 minutes) - Charge (6 minutes) \
   lost_capacity: 0.048 | Q30: 0.115 | objective: 0.139
    
2. **"Pulse Rest"**: Charge - Rest - Charge - Rest - Charge (6 minutes/each) \
   lost_capacity: 0.861 | Q30: 0.125 | objective: 0.126
    
Both of these strategies are valid solutions that have been discussed in the literature and present a tradeoff between prioritizing more capacity versus low degradation.  

All 10 solutions the BO suggested can be found in the cell below. 

---

## Conclusions & Next Steps

This Bayesian optimization algorithm demonstrates that a 5-step charging protocol can be optimized effectively and the optimal strategy produced by this algorithm is similar to the 3-step BO ("charge - rest - charge") and performs slightly better. After running the solutions at 5% below and above the sugggested ones (within the 3C limit), the objective wasn't really improved and so a local minimum was indeed reached. 

Although the exact solutions suggested by this 5-step BO were less consistent than the 3-step protocol, the algorithm frequently presents the user with two valid designs found in the literature: "long rest" and "pulse rest". This decrease in consistency can be simply due to the presence of more valid solutions because of the higher number of combinations available ($N^3$ vs $N^5$ where N = # of possible C-rates at each step).

While this notebook does show the effectiveness of BO in optimizing fast-charging protocols, it is important to note that the exact solution relies on the accuracy of the P2D model. In real situations the model should be parameterized to the actual cell used and a thermal model should be included. 

**Potential next steps**:
- N-step charging protocol (instead of just 5)
- Time of each step as a parameter (instead of 6 minutes for all)
- Both N-step + t-step as parameters (most general charging profile)

---

## Appendix (All BO Solutions)

In [7]:
beta = 0.008

print("Soln 1: \n")

soln = np.array([3,0,0,0,2.5187064929422838])
run_P2D(soln,beta)

print("\nSoln 2: \n")

new_soln = np.array([3,0,0,0,2.418492956058545])
run_P2D(new_soln,beta)

print("\nSoln 3: \n")

new_soln = np.array([3,0,3,0,2.4632001141882274])
run_P2D(new_soln,beta)

print("\nSoln 4: \n")

new_soln = np.array([3,0,3,0,2.426067620768284])
run_P2D(new_soln,beta)

print("\nSoln 5: \n")

new_soln = np.array([3,0,0,3,2.0565040778528902])
run_P2D(new_soln,beta)

print("\nSoln 6: \n")

new_soln = np.array([3,0,2.7676139732376295,0,2.4432537834956873])
run_P2D(new_soln,beta)

print("\nSoln 7 \n")

new_soln = np.array([3,3,2.6397239560705907,0,3])
run_P2D(new_soln,beta)

print("\nSoln 8: \n")

new_soln = np.array([3,0,0,0,2.0675333029974063])
run_P2D(new_soln,beta)

print("\nSoln 9: \n")

new_soln = np.array([2.7463103000376803,0,0,0,2.6268851401303963])
run_P2D(new_soln,beta)

print("\nSoln 10: \n")

new_soln = np.array([3,0,3,0,3])
run_P2D(new_soln,beta)

Soln 1: 



At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.048379699671884005, 'Q30': 0.11497305193629752, 'objective': 0.13920245178838592}

Soln 2: 



At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.03902281590475144, 'Q30': 0.1128852699178863, 'objective': 0.13883414016359105}

Soln 3: 



...

At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.0478128870768618, 'Q30': 0.11194157167016823, 'objective': 0.13626525224379174}

Soln 10: 



At t = 472.929 and h = 2.38432e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 232.929 and h = 1.9824e-11, the corrector convergence failed repeatedly or with |h| = hmin.


{'lost_capacity': 0.8610813906627672, 'Q30': 0.12499999999999993, 'objective': 0.12619652998908498}


{'objective': 0.12619652998908498}